In [1]:
import pandas as pd
import numpy as np

# Operating System
import os

# https requests
import requests

import matplotlib.pyplot as plt
%matplotlib inline

idx = pd.IndexSlice

# World Population  

[This UN source](https://data.un.org/Data.aspx?d=PopDiv&f=variableID%3a12%3btimeID%3a83%2c84%3bvarID%3a2&c=2,4,6,7&s=_crEngNameOrderBy:asc,_timeEngNameOrderBy:desc,_varEngNameOrderBy:asc&v=1#PopDiv) was used.

In [2]:
filename = "./data/UNdata_Export_20200324_225732447.csv"

# read csv data file into a DataFrame
df_population = pd.read_csv(filename)

# Tidy the data and keep just the values we will need
df_population.columns = ['country', 'year', 'variant', 'million_people']
df_population['million_people'] = df_population['million_people']/1000
df_population['country'] = df_population['country'].str.lower()
df_population = df_population.loc[df_population.year == 2019]
df_population = df_population.drop(columns=['year', 'variant'])
df_population = df_population.drop_duplicates()
df_population = df_population.reset_index(drop=True)

# change country names to match the COVID database
replacement_dict = {'bolivia (plurinational state of)':'bolivia',
                    'iran (islamic republic of)':'iran',
                    'republic of korea':'korea, south' ,
                    'russian federation':'russia' ,
                    'syrian arab republic':'syria' ,
                    'united states of america':'us' ,
                    'venezuela (bolivarian republic of)':'venezuela' ,
                    'viet nam':'vietnam'}

df_population['country'] = df_population['country'].replace(to_replace=replacement_dict)

# validate no duplicate values
assert df_population.country.duplicated().sum() == 0

df_population = df_population.set_index('country').sort_index()

In [3]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 282 entries, afghanistan to zimbabwe
Data columns (total 1 columns):
million_people    282 non-null float64
dtypes: float64(1)
memory usage: 4.4+ KB


In [4]:
df_population.head()

,million_people
country,
afghanistan,38.041754
africa,1308.064195
albania,2.880917
algeria,43.053054
american samoa,0.055312


In [5]:
df_population.to_pickle('./data/df_population.pkl')

# Import the COVID data

### Data Sources  


* https://github.com/datasets/covid-19
* https://github.com/CSSEGISandData/COVID-19
* https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide

In [6]:
# We will be using the first data source

url = "https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv"
r = requests.get(url)

filename = "./data/covid_panel_data_aggregation.csv"

f = open(filename, "w") 
f.write(r.text) 
f.close()

# read csv data file into a DataFrame
df_original = pd.read_csv(filename)

In [7]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21648 entries, 0 to 21647
Data columns (total 8 columns):
Date              21648 non-null object
Country/Region    21648 non-null object
Province/State    6724 non-null object
Lat               21648 non-null float64
Long              21648 non-null float64
Confirmed         21566 non-null float64
Recovered         20500 non-null float64
Deaths            21566 non-null float64
dtypes: float64(5), object(3)
memory usage: 1.3+ MB


### confirm if database has been updated

In [8]:
df_original.tail()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
21643,2020-04-08,Zimbabwe,NaN,-20.0,30.0,11.0,0.0,3.0
21644,2020-04-09,Zimbabwe,NaN,-20.0,30.0,11.0,0.0,3.0
21645,2020-04-10,Zimbabwe,NaN,-20.0,30.0,13.0,0.0,3.0
21646,2020-04-11,Zimbabwe,NaN,-20.0,30.0,14.0,0.0,3.0
21647,2020-04-12,Zimbabwe,NaN,-20.0,30.0,14.0,0.0,3.0


### Buid main DataFrame

In [9]:
# Tidy DataFrame

def tidy_df(df_original, df_population=None):
    
    df_tidy = df_original.copy()
    
    # drop rows in which any of the values we will use is NaN
    df_tidy = df_tidy.dropna(how='any', subset=['Confirmed', 'Deaths'])
    
    # simplify 'Country/Region' and Province/State'    
    df_tidy = df_tidy.rename(columns = {'Country/Region': 'country',
                                        'Province/State': 'state',
                                        'Confirmed': 'total_infections',
                                        'Deaths': 'total_deaths'
                                       })
    
    # conver column names to lower case
    df_tidy.columns = df_tidy.columns.str.lower()
    
    # Convert Date from string to datetime
    df_tidy["date"] = pd.to_datetime(df_tidy.date)
    
    # Convert string values to lower case
    df_tidy['country'] = df_tidy.country.str.lower()
    df_tidy['state'] = df_tidy.state.str.lower()
    
    # Fill string nans with ''
    df_tidy['state'] = df_tidy.state.fillna('')
    
    # If country equals the state (e.g., france, united kingdom, us,...) remove the state
    df_tidy.loc[df_tidy.country==df_tidy.state, 'state'] = ""
    
    
    # Merge COVID with population DataFrames    
    # df_tidy = pd.merge(df_tidy, df_population, how='inner', on='country', validate='many_to_one')
    
    
    # Set new index
    df_tidy = df_tidy.set_index(keys=['country', 'state', 'date']).sort_index()
    
    # Build variables to use afterward
#     df_tidy['currently infected'] = df_tidy['confirmed'] - df_tidy['recovered']    
#     df_tidy['currently infected increase'] = df_tidy['currently infected'].diff()
#     df_tidy['deaths increase'] = df_tidy['deaths'].diff()
#     df_tidy['new_infections_growth_rate'] = df_tidy['new_infections'].pct_change().mul(100)
    
    df_tidy['new_infections'] = df_tidy['total_infections'].diff()
    df_tidy['new_infections_increase'] = df_tidy['new_infections'].diff()  
    
    df_tidy['new_deaths'] = df_tidy['total_deaths'].diff()
    
    #df_tidy = df_tidy[['lat', 'long', 'total_infections', 'new_infections', 'new_infections_increase', 'recovered', 'total_deaths', 'new_deaths']]
    df_tidy = df_tidy[['lat', 'long', 'total_infections', 'new_infections', 'new_infections_increase', 'total_deaths', 'new_deaths']]
    
    return df_tidy


In [10]:
df_main = tidy_df(df_original)

In [11]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21566 entries, (afghanistan, , 2020-01-22 00:00:00) to (zimbabwe, , 2020-04-12 00:00:00)
Data columns (total 7 columns):
lat                        21566 non-null float64
long                       21566 non-null float64
total_infections           21566 non-null float64
new_infections             21565 non-null float64
new_infections_increase    21564 non-null float64
total_deaths               21566 non-null float64
new_deaths                 21565 non-null float64
dtypes: float64(7)
memory usage: 1.2+ MB


In [12]:
# df_main.index.get_level_values('state').drop_duplicates()

In [13]:
df_main.loc[idx["portugal", :,"2020-03-03":], :]

lat    long  total_infections  new_infections  \
country  state date                                                            
portugal       2020-03-03  39.3999 -8.2245               2.0             0.0   
               2020-03-04  39.3999 -8.2245               5.0             3.0   
               2020-03-05  39.3999 -8.2245               8.0             3.0   
               2020-03-06  39.3999 -8.2245              13.0             5.0   
               2020-03-07  39.3999 -8.2245              20.0             7.0   
               2020-03-08  39.3999 -8.2245              30.0            10.0   
               2020-03-09  39.3999 -8.2245              30.0             0.0   
               2020-03-10  39.3999 -8.2245              41.0            11.0   
               2020-03-11  39.3999 -8.2245              59.0            18.0   
               2020-03-12  39.3999 -8.2245              59.0             0.0   
               2020-03-13  39.3999 -8.2245             112.0            53.0   
               2020-03-14  39.3999 -8.2245             169.0            57.0   
               2020-03-15  39.3999 -8.2245             245.0            76.0   
               2020-03-16  39.3999 -8.2245             331.0            86.0   
               2020-03-17  39.3999 -8.2245             448.0           117.0   
               2020-03-18  39.3999 -8.2245             448.0             0.0   
               2020-03-19  39.3999 -8.2245             785.0           337.0   
               2020-03-20  39.3999 -8.2245            1020.0           235.0   
               2020-03-21  39.3999 -8.2245            1280.0           260.0   
               2020-03-22  39.3999 -8.2245            1600.0           320.0   
               2020-03-23  39.3999 -8.2245            2060.0           460.0   
               2020-03-24  39.3999 -8.2245            2362.0           302.0   
               2020-03-25  39.3999 -8.2245            2995.0           633.0   
               2020-03-26  39.3999 -8.2245            3544.0           549.0   
               2020-03-27  39.3999 -8.2245            4268.0           724.0   
               2020-03-28  39.3999 -8.2245            5170.0           902.0   
               2020-03-29  39.3999 -8.2245            5962.0           792.0   
               2020-03-30  39.3999 -8.2245            6408.0           446.0   
               2020-03-31  39.3999 -8.2245            7443.0          1035.0   
               2020-04-01  39.3999 -8.2245            8251.0           808.0   
               2020-04-02  39.3999 -8.2245            9034.0           783.0   
               2020-04-03  39.3999 -8.2245            9886.0           852.0   
               2020-04-04  39.3999 -8.2245           10524.0           638.0   
               2020-04-05  39.3999 -8.2245           11278.0           754.0   
               2020-04-06  39.3999 -8.2245           11730.0           452.0   
               2020-04-07  39.3999 -8.2245           12442.0           712.0   
               2020-04-08  39.3999 -8.2245           13141.0           699.0   
               2020-04-09  39.3999 -8.2245           13956.0           815.0   
               2020-04-10  39.3999 -8.2245           15472.0          1516.0   
               2020-04-11  39.3999 -8.2245           15987.0           515.0   
               2020-04-12  39.3999 -8.2245           16585.0           598.0   

                           new_infections_increase  total_deaths  new_deaths  
country  state date                                                           
portugal       2020-03-03                     -2.0           0.0         0.0  
               2020-03-04                      3.0           0.0         0.0  
               2020-03-05                      0.0           0.0         0.0  
               2020-03-06                      2.0           0.0         0.0  
               2020-03-07                      2.0           0.0         0.0  
               2020-03-08            

In [14]:
df_main.to_pickle('./data/df_main.pkl')

In [15]:
df_main.to_excel('./data/df_main.xlsx')